In [1]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [2]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [4]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [5]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [6]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [7]:
## 
n='Input_A3_016' # 改名稱
##
dire= ['./Data_Science_2020_imdb/BayesianRidge_Pre/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A3_016
./Data_Science_2020_imdb/BayesianRidge_Pre/
feature number: 2
train_rmse: 0.009088453741332783
test_rmse: 0.00957901846670066
test_r2: 0.511046515502869
feature number: 3
train_rmse: 0.008860190611679523
test_rmse: 0.009298118184459478
test_r2: 0.5391979717313645
feature number: 4
train_rmse: 0.0087536652857226
test_rmse: 0.00937913557254171
test_r2: 0.530983646861815
feature number: 5
train_rmse: 0.008598239606244286
test_rmse: 0.009315598551596312
test_r2: 0.5368512592500971
feature number: 6
train_rmse: 0.008586983789634888
test_rmse: 0.00931659548181627
test_r2: 0.5367512610452647
feature number: 7
train_rmse: 0.008568542184446234
test_rmse: 0.00933426012731697
test_r2: 0.5351510154853656
feature number: 8
train_rmse: 0.008505921572252061
test_rmse: 0.009249846146396823
test_r2: 0.5436609914464399
feature number: 9
train_rmse: 0.008478723252556662
test_rmse: 0.009325145846810408
test_r2: 0.5360267151276609
feature number: 10
train_rmse: 0.008468922598711204
test_rmse: 

train_rmse: 0.008415895241937763
test_rmse: 0.0093952501789489
test_r2: 0.5285792461814429
feature number: 15
train_rmse: 0.00840609478466989
test_rmse: 0.009451018190997742
test_r2: 0.5232168244581248
feature number: 16
train_rmse: 0.008365752659443958
test_rmse: 0.009464426713994594
test_r2: 0.5218851729559253
feature number: 17
train_rmse: 0.008315384152172188
test_rmse: 0.009505313080868875
test_r2: 0.5175407616965517
feature number: 18
train_rmse: 0.008275876559421769
test_rmse: 0.009536190313835936
test_r2: 0.514164873862391
feature number: 19
train_rmse: 0.0081915834380057
test_rmse: 0.009533039992027505
test_r2: 0.5139846873047282
feature number: 20
train_rmse: 0.008183580792679482
test_rmse: 0.009526212286483579
test_r2: 0.5146642372608602
Test_rmse_min = 1.fea_num: 8  2.rmse: 0.009249846146396823
Test_r2_max = 1. fea_num: 8  2.r2: 0.5436609914464399
./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.009088453741332783
test_rmse: 0.0095790184667

train_rmse: 0.008568542184446234
test_rmse: 0.00933426012731697
test_r2: 0.5351510154853656
feature number: 8
train_rmse: 0.008505921572252061
test_rmse: 0.009249846146396823
test_r2: 0.5436609914464399
feature number: 9
train_rmse: 0.008478723252556662
test_rmse: 0.009325145846810408
test_r2: 0.5360267151276609
feature number: 10
train_rmse: 0.008468922598711204
test_rmse: 0.009326918970649956
test_r2: 0.5359051547015179
feature number: 11
train_rmse: 0.008449176188472744
test_rmse: 0.009306672284633024
test_r2: 0.5378760919032175
feature number: 12
train_rmse: 0.008444372825714254
test_rmse: 0.009326805404079144
test_r2: 0.5357116860456845
feature number: 13
train_rmse: 0.008439151163841373
test_rmse: 0.009360106120592303
test_r2: 0.5325078139274624
feature number: 14
train_rmse: 0.008415895241937763
test_rmse: 0.0093952501789489
test_r2: 0.5285792461814429
feature number: 15
train_rmse: 0.00840609478466989
test_rmse: 0.009451018190997742
test_r2: 0.5232168244581248
feature number: 1

# 可以修改第一層的神經元數量

In [8]:
#
def build_model():
    model = models.Sequential()
#model.add(layers.Dense(7, activation='sigmoid',input_shape=(len(X[0]),)))
    model.add(layers.Dense(7, activation='sigmoid',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [9]:
fea_num=[3,8] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/BayesianRidge_Pre/Input_A3_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 3
test_rmse_ave: 0.0094717179809877
[0.009636573802759238, 0.008014182217342061, 0.00966471961884376, 0.009738198622615749, 0.008116931025665911, 0.008020510682098777, 0.009883815599861985, 0.009970766368278918, 0.010700100122100516, 0.010971381750310078]


test_r2_ave: 0.5095226056992784
[0.5225449419899157, 0.6331750213611342, 0.503561285906429, 0.495758776411943, 0.6156867994115849, 0.6378093370204516, 0.4514022301529963, 0.428080664133654, 0.400465144051929, 0.406741856552746]


feature number: 8
test_rmse_ave: 0.010065135529581675
[0.012444732892261497, 0.008115522033061207, 0.009793921890505264, 0.011538841241648161, 0.008210349802986858, 0.008155894972271639, 0.010399331526230923, 0.010632783922503017, 0.010381108541351082, 0.010978868472997108]


test_r2_ave: 0.442599294935733
[0.20373366657332703, 0.6238393191521183, 0.4901993375159277, 0.2920452701261873, 0.6067896756334947, 0.6254787569243927, 0.3926827506814061, 0.34961326333868725, 0.43567899179525493, 0.40

In [10]:
fea_num=[4,5,6,7]   # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/Input_A3_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.009848158164864953
[0.010031288888557425, 0.008849302703223517, 0.01117227060548446, 0.010504184594095661, 0.00845435574085999, 0.00880818261323631, 0.009625685302204107, 0.0106702048448066, 0.010006678844885921, 0.010359427511295542]


test_r2_ave: 0.4702981808594336
[0.48263068146708743, 0.552741526121951, 0.3366083269417044, 0.4133139230831705, 0.5830704978464452, 0.563176668171722, 0.4796829142557424, 0.3450272773334111, 0.47565313244190843, 0.47107686093119283]


feature number: 5
test_rmse_ave: 0.009678281044744164
[0.010763246040451438, 0.008376323616887301, 0.010208935160881269, 0.010568165884023577, 0.00801745814531195, 0.008300927667903184, 0.0098799855106961, 0.010674788091038263, 0.010483319726729941, 0.009509660603518613]


test_r2_ave: 0.48680558621753744
[0.40437388507209016, 0.5992741385423501, 0.44607876507710453, 0.4061451131842597, 0.6250485876409979, 0.6120404327063764, 0.4518273233244392, 0.3444644866428316, 0.42451180789877285, 0

In [ ]:
fea_num=[4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/Input_A3_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4


In [ ]:
fea_num=[4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/Input_A3_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/Input_A3_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/Input_A3_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/Input_A3_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/Input_A3_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')